# Sentiment Classification

For this I will be using the "SemEval 2017 task 4" My focus is particularly on Subtask A, i.e. classifying the overall sentiment of a tweet as positive, negative or neutral.



#### Import necessary packages


In [1]:
# Import necessary packages
import pandas as pd
import numpy as np
import re
import nltk
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.stem import WordNetLemmatizer
from nltk import pos_tag
from sklearn.feature_extraction.text import TfidfVectorizer,CountVectorizer
from sklearn.svm import SVC
from sklearn.naive_bayes import MultinomialNB
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import classification_report
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import LabelEncoder
from torch.utils.data import TensorDataset, DataLoader
import os

import torch
import torch.nn as nn
from torchtext.vocab import Vectors
import torch.optim as optim

In [2]:
# Define test sets
testsets = ['twitter-test1.txt', 'twitter-test2.txt', 'twitter-test3.txt']

In [3]:
# Evaluation code for the test sets
def read_test(testset):
    '''
    readin the testset and return a dictionary
    :param testset: str, the file name of the testset to compare
    '''
    id_gts = {}
    with open(testset, 'r', encoding='utf8') as fh:
        for line in fh:
            fields = line.split('\t')
            tweetid = fields[0]
            gt = fields[1]

            id_gts[tweetid] = gt

    return id_gts


def confusion(id_preds, testset, classifier):
    '''
    print the confusion matrix of {'positive', 'netative'} between preds and testset
    :param id_preds: a dictionary of predictions formated as {<tweetid>:<sentiment>, ... }
    :param testset: str, the file name of the testset to compare
    :classifier: str, the name of the classifier
    '''
    id_gts = read_test(testset)

    gts = []
    for m, c1 in id_gts.items():
        if c1 not in gts:
            gts.append(c1)

    gts = ['positive', 'negative', 'neutral']

    conf = {}
    for c1 in gts:
        conf[c1] = {}
        for c2 in gts:
            conf[c1][c2] = 0

    for tweetid, gt in id_gts.items():
        if tweetid in id_preds:
            pred = id_preds[tweetid]
        else:
            pred = 'neutral'
        conf[pred][gt] += 1

    print(''.ljust(12) + '  '.join(gts))

    for c1 in gts:
        print(c1.ljust(12), end='')
        for c2 in gts:
            if sum(conf[c1].values()) > 0:
                print('%.3f     ' % (conf[c1][c2] / float(sum(conf[c1].values()))), end='')
            else:
                print('0.000     ', end='')
        print('')

    print('')


def evaluate(id_preds, testset, classifier):
    '''
    print the macro-F1 score of {'positive', 'netative'} between preds and testset
    :param id_preds: a dictionary of predictions formated as {<tweetid>:<sentiment>, ... }
    :param testset: str, the file name of the testset to compare
    :classifier: str, the name of the classifier
    '''
    id_gts = read_test(testset)

    acc_by_class = {}
    for gt in ['positive', 'negative', 'neutral']:
        acc_by_class[gt] = {'tp': 0, 'fp': 0, 'tn': 0, 'fn': 0}

    catf1s = {}

    ok = 0
    for tweetid, gt in id_gts.items():
        if tweetid in id_preds:
            pred = id_preds[tweetid]
        else:
            pred = 'neutral'

        if gt == pred:
            ok += 1
            acc_by_class[gt]['tp'] += 1
        else:
            acc_by_class[gt]['fn'] += 1
            acc_by_class[pred]['fp'] += 1

    catcount = 0
    itemcount = 0
    macro = {'p': 0, 'r': 0, 'f1': 0}
    micro = {'p': 0, 'r': 0, 'f1': 0}
    semevalmacro = {'p': 0, 'r': 0, 'f1': 0}

    microtp = 0
    microfp = 0
    microtn = 0
    microfn = 0
    for cat, acc in acc_by_class.items():
        catcount += 1

        microtp += acc['tp']
        microfp += acc['fp']
        microtn += acc['tn']
        microfn += acc['fn']

        p = 0
        if (acc['tp'] + acc['fp']) > 0:
            p = float(acc['tp']) / (acc['tp'] + acc['fp'])

        r = 0
        if (acc['tp'] + acc['fn']) > 0:
            r = float(acc['tp']) / (acc['tp'] + acc['fn'])

        f1 = 0
        if (p + r) > 0:
            f1 = 2 * p * r / (p + r)

        catf1s[cat] = f1

        n = acc['tp'] + acc['fn']

        macro['p'] += p
        macro['r'] += r
        macro['f1'] += f1

        if cat in ['positive', 'negative']:
            semevalmacro['p'] += p
            semevalmacro['r'] += r
            semevalmacro['f1'] += f1

        itemcount += n

    micro['p'] = float(microtp) / float(microtp + microfp)
    micro['r'] = float(microtp) / float(microtp + microfn)
    micro['f1'] = 2 * float(micro['p']) * micro['r'] / float(micro['p'] + micro['r'])

    semevalmacrof1 = semevalmacro['f1'] / 2

    print(testset + ' (' + classifier + '): %.3f' % semevalmacrof1)

#### Load training set, dev set and testing set
Here, you need to load the training set, the development set and the test set. For better classification results, you may need to preprocess tweets before sending them to the classifiers.

In [4]:
def load_pandas_dataframe(filepath):
    df = pd.read_csv(filepath, sep='\t', header=None, names=['tweet_id', 'sentiment', 'tweet-text'])
    return df

# Loading the training set, development set and 3 test sets
train_path = os.path.join(os.getcwd(), 'semeval-tweets', 'twitter-training-data.txt')
dev_path = os.path.join(os.getcwd(), 'semeval-tweets', 'twitter-dev-data.txt')
test1_path = os.path.join(os.getcwd(), 'semeval-tweets', 'twitter-test1.txt')
test2_path = os.path.join(os.getcwd(), 'semeval-tweets', 'twitter-test2.txt')
test3_path = os.path.join(os.getcwd(), 'semeval-tweets', 'twitter-test3.txt')

train_df = load_pandas_dataframe(train_path)
dev_df = load_pandas_dataframe(dev_path)
test1_df = load_pandas_dataframe(test1_path)
test2_df = load_pandas_dataframe(test2_path)
test3_df = load_pandas_dataframe(test3_path)

In [5]:
# Download NLTK resources
nltk.download('punkt')
nltk.download('wordnet')
nltk.download('stopwords')
nltk.download('averaged_perceptron_tagger')

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\VatsaL\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\VatsaL\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\VatsaL\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     C:\Users\VatsaL\AppData\Roaming\nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!


True

In [6]:
# Preprocessing
def preprocess_text(text):
    # Convert to lowercase
    text = text.lower()
    # Remove URLs
    text = re.sub(r'http\S+|www\S+', '', text)
    # Remove hashtags
    text = re.sub(r'#\w+', '', text)
    # Remove user mentions
    text = re.sub(r'@\w+', '', text)
    # Remove punctuation
    text = re.sub(r'[^\w\s]', '', text)
    # Remove numbers
    text = re.sub(r'\d+', '', text)
    # Tokenization
    tokens = word_tokenize(text)
    # Remove stopwords
    stop_words = set(stopwords.words('english'))
    tokens = [word for word in tokens if word not in stop_words]
    # POS tagging
    tagged_tokens = pos_tag(tokens)
    # Lemmatization with POS tagging
    lemmatizer = WordNetLemmatizer()
    lemmatized_tokens = []
    for word, tag in tagged_tokens:
        pos = 'n'  # default to noun if not found
        if tag.startswith('J'):
            pos = 'a'  # adjective
        elif tag.startswith('V'):
            pos = 'v'  # verb
        elif tag.startswith('R'):
            pos = 'r'  # adverb
        lemmatized_tokens.append(lemmatizer.lemmatize(word, pos))
    # Join tokens back into text
    preprocessed_text = ' '.join(lemmatized_tokens)
    return preprocessed_text

In [7]:
train_df['tweet-text'] = train_df['tweet-text'].apply(preprocess_text)
dev_df['tweet-text'] = dev_df['tweet-text'].apply(preprocess_text)
test1_df['tweet-text'] = test1_df['tweet-text'].apply(preprocess_text)
test2_df['tweet-text'] = test2_df['tweet-text'].apply(preprocess_text)
test3_df['tweet-text'] = test3_df['tweet-text'].apply(preprocess_text)


In [8]:
X_train = train_df['tweet-text']
y_train = train_df['sentiment']

X_dev = dev_df['tweet-text']
y_dev = dev_df['sentiment'] 

X_test1 = test1_df['tweet-text']
y_test1 = test1_df['sentiment']

X_test2 = test2_df['tweet-text']
y_test2 = test2_df['sentiment']

X_test3 = test3_df['tweet-text']
y_test3 = test3_df['sentiment']

#### Build sentiment classifiers
You need to create your own classifiers (at least 3 classifiers). For each classifier, you can choose between the bag-of-word features and the word-embedding-based features. Each classifier has to be evaluated over 3 test sets. Make sure your classifier produce consistent performance across the test sets. Marking will be based on the performance over all 5 test sets (2 of them are not provided to you).

In [9]:
# Buid traditional sentiment classifiers. An example classifier name 'svm' is given
# in the code below. You should replace the other two classifier names
# with your own choices. For features used for classifier training, 
# the 'bow' feature is given in the code. But you could also explore the 
# use of other features.
for classifier in ['maxent', 'naive_bayes', 'svm', 'LSTM']:
    for features in ['bow', 'glove']:
        # Skeleton: Creation and training of the classifiers
        if classifier == 'maxent':
            # write the svm classifier here
            print('Training ' + classifier)
            if features == 'bow':
                pipeline_maxent_clf = Pipeline([
                    ('tfidf', TfidfVectorizer(ngram_range = (1,3))),
                    ('clf', LogisticRegression(solver='lbfgs', C=1, max_iter=100, penalty='l2'))
                   ])
            elif features == 'glove':
                pass
                pipeline_maxent_clf.fit(X_train,y_train)
                y_pred_maxent = pipeline_maxent_clf.predict(X_dev)
                print(classification_report(y_dev, y_pred_maxent))
                dev_tweet_ids = dev_df['tweet_id'].tolist()
                predictions_dev_maxent = {tweet_id: pred for tweet_id, pred in zip(dev_tweet_ids, y_pred_maxent)}
                predictions_dev_maxent = {str(tweet_id): pred for tweet_id, pred in zip(dev_tweet_ids, y_pred_maxent)}
                evaluate(predictions_dev_maxent, dev_path, 'MAXENT')

                test1_tweet_ids = test1_df['tweet_id'].tolist()
                predictions_test1_maxent = {tweet_id: pred for tweet_id, pred in zip(test1_tweet_ids, y_pred_maxent)}
                predictions_test1_maxent = {str(tweet_id): pred for tweet_id, pred in zip(test1_tweet_ids, y_pred_maxent)}

                test2_tweet_ids = test2_df['tweet_id'].tolist()
                predictions_test2_maxent = {tweet_id: pred for tweet_id, pred in zip(test2_tweet_ids, y_pred_maxent)}
                predictions_test2_maxent = {str(tweet_id): pred for tweet_id, pred in zip(test2_tweet_ids, y_pred_maxent)}

                test3_tweet_ids = test3_df['tweet_id'].tolist()
                predictions_test3_maxent = {tweet_id: pred for tweet_id, pred in zip(test3_tweet_ids, y_pred_maxent)}
                predictions_test3_maxent = {str(tweet_id): pred for tweet_id, pred in zip(test3_tweet_ids, y_pred_maxent)}

                evaluate(predictions_test1_maxent, test1_path, 'MAXENT')
                evaluate(predictions_test2_maxent, test2_path, 'MAXENT')
                evaluate(predictions_test3_maxent, test3_path, 'MAXENT')
                
        elif classifier == 'naive_bayes':
            # write the classifier 2 here
            print('Training ' + classifier)
            if features == 'bow':
                pipeline_NB_clf = Pipeline([
                    ('tfidf', TfidfVectorizer(ngram_range = (1,3))),
                    ('clf', MultinomialNB(alpha =0.1))
                   ])
            elif features == 'glove':
                pass
                pipeline_NB_clf.fit(X_train,y_train)
                y_pred_NB = pipeline_NB_clf.predict(X_dev)
                print(classification_report(y_dev, y_pred_NB))
                dev_tweet_ids = dev_df['tweet_id'].tolist()
                predictions_dev_NB = {tweet_id: pred for tweet_id, pred in zip(dev_tweet_ids, y_pred_NB)}
                predictions_dev_NB = {str(tweet_id): pred for tweet_id, pred in zip(dev_tweet_ids, y_pred_NB)}
                evaluate(predictions_dev_NB, dev_path, 'NB')
                test1_tweet_ids = test1_df['tweet_id'].tolist()
                predictions_test1_NB = {tweet_id: pred for tweet_id, pred in zip(test1_tweet_ids, y_pred_NB)}
                predictions_test1_NB = {str(tweet_id): pred for tweet_id, pred in zip(test1_tweet_ids, y_pred_NB)}

                test2_tweet_ids = test2_df['tweet_id'].tolist()
                predictions_test2_NB = {tweet_id: pred for tweet_id, pred in zip(test2_tweet_ids, y_pred_NB)}
                predictions_test2_NB = {str(tweet_id): pred for tweet_id, pred in zip(test2_tweet_ids, y_pred_NB)}

                test3_tweet_ids = test3_df['tweet_id'].tolist()
                predictions_test3_NB = {tweet_id: pred for tweet_id, pred in zip(test3_tweet_ids, y_pred_NB)}
                predictions_test3_NB = {str(tweet_id): pred for tweet_id, pred in zip(test3_tweet_ids, y_pred_NB)}

                evaluate(predictions_test1_NB, test1_path, 'NB')
                evaluate(predictions_test2_NB, test2_path, 'NB')
                evaluate(predictions_test3_NB, test3_path, 'NB')
        elif classifier == 'svm':
            # write the classifier 3 here
            print('Training ' + classifier)
            if features == 'bow':
                pipeline_SVM_clf = Pipeline([
                    ('tfidf', TfidfVectorizer(ngram_range = (1,3))),
                    ('clf', SVC(kernel = 'linear', C = 1 ))
                   ])
            elif features == 'glove':
                pass
                pipeline_SVM_clf.fit(X_train,y_train)
                y_pred_SVM = pipeline_SVM_clf.predict(X_dev)
                print(classification_report(y_dev, y_pred_SVM))

                dev_tweet_ids = dev_df['tweet_id'].tolist()
                predictions_dev_SVM = {tweet_id: pred for tweet_id, pred in zip(dev_tweet_ids, y_pred_SVM)}
                predictions_dev_SVM = {str(tweet_id): pred for tweet_id, pred in zip(dev_tweet_ids, y_pred_SVM)}
                evaluate(predictions_dev_SVM, dev_path, 'SVM')

                test1_tweet_ids = test1_df['tweet_id'].tolist()
                predictions_test1_SVM = {tweet_id: pred for tweet_id, pred in zip(test1_tweet_ids, y_pred_SVM)}
                predictions_test1_SVM = {str(tweet_id): pred for tweet_id, pred in zip(test1_tweet_ids, y_pred_SVM)}

                test2_tweet_ids = test2_df['tweet_id'].tolist()
                predictions_test2_SVM = {tweet_id: pred for tweet_id, pred in zip(test2_tweet_ids, y_pred_SVM)}
                predictions_test2_SVM = {str(tweet_id): pred for tweet_id, pred in zip(test2_tweet_ids, y_pred_SVM)}

                test3_tweet_ids = test3_df['tweet_id'].tolist()
                predictions_test3_SVM = {tweet_id: pred for tweet_id, pred in zip(test3_tweet_ids, y_pred_SVM)}
                predictions_test3_SVM = {str(tweet_id): pred for tweet_id, pred in zip(test3_tweet_ids, y_pred_SVM)}

                evaluate(predictions_test1_SVM, test1_path, 'SVM')
                evaluate(predictions_test2_SVM, test2_path, 'SVM')
                evaluate(predictions_test3_SVM, test3_path, 'SVM')
        elif classifier == 'LSTM':
            # write the classifier 4 here
            print('Training' + classifier)
            if features == 'bow':
                pass
            elif features == 'glove':
                def load_glove_embeddings(path):
                    embeddings_dict = {}
                    with open(path, 'r', encoding="utf8") as f:
                        for line in f:
                            values = line.split()
                            word = values[0]
                            vector = np.asarray(values[1:], "float32")
                            embeddings_dict[word] = vector
                    return embeddings_dict

                glove_path = os.path.join(os.getcwd(), "glove.6B", "glove.6B.100d.txt")
                glove_embeddings = load_glove_embeddings(glove_path)
                
                
                X_train_tokens = X_train.apply(lambda x: word_tokenize(x.lower()))
                X_dev_tokens = X_dev.apply(lambda x: word_tokenize(x.lower()))

                word_index = {}
                current_index = 1
                for tokens in X_train_tokens:
                    for token in tokens:
                        if token not in word_index:
                            word_index[token] = current_index
                            current_index += 1

                X_train_seq = [[word_index[token] for token in tokens if token in word_index] for tokens in X_train_tokens]
                X_dev_seq = [[word_index[token] for token in tokens if token in word_index] for tokens in X_dev_tokens]

                max_length = max(max(len(seq) for seq in X_train_seq), max(len(seq) for seq in X_dev_seq))
                
                def pading_sequences(sequences, maxlen):
                    padded_sequences = np.zeros((len(sequences), maxlen), dtype=int)
                    for i, seq in enumerate(sequences):
                        len_seq = len(seq)
                        if len_seq > maxlen:
                            padded_sequences[i, :] = seq[:maxlen]
                        elif len_seq > 0:
                            padded_sequences[i, -len_seq:] = seq
                    return padded_sequences
                
                X_train = pading_sequences(X_train_seq, maxlen=max_length)
                X_dev = pading_sequences(X_dev_seq, maxlen=max_length)
                
                vocab_size = len(word_index) + 1
                embedding_dim = 100

                embedding_matrix = np.zeros((vocab_size, embedding_dim))

                for word, i in word_index.items():
                    embedding_vector = glove_embeddings.get(word)
                    if embedding_vector is not None:
                        embedding_matrix[i] = embedding_vector
                    
                

                class LSTMClassifier(nn.Module):
                  def __init__(self, vocab_size, embedding_dim, hidden_dim, output_dim, embedding_matrix):
                    super(LSTMClassifier, self).__init__()

                    # Hyperparameters
                    self.vocab_size = vocab_size
                    self.embedding_dim = embedding_dim
                    self.hidden_dim = hidden_dim
                    self.output_dim = output_dim

                    # Embedding layer
                    self.embedding = nn.Embedding(vocab_size, embedding_dim)
                    self.embedding.weight = nn.Parameter(torch.tensor(embedding_matrix, dtype=torch.float32))
                    self.embedding.weight.requires_grad = False  # Freeze pre-trained embeddings

                    # LSTM layer (batch_first=True)
                    self.lstm = nn.LSTM(embedding_dim, hidden_dim, batch_first=True)

                    # Dense layer for output
                    self.fc = nn.Linear(hidden_dim, output_dim)

                  def forward(self, text):
                    # Get embeddings from input text
                    embedded = self.embedding(text)

                    # Pass embeddings through LSTM
                    _, (hidden, _) = self.lstm(embedded)

                    # Get last hidden state
                    hidden = hidden[-1]

                    # Pass last hidden state through dense layer
                    output = self.fc(hidden)

                    return output


                lstm_model = LSTMClassifier(vocab_size, embedding_dim, hidden_dim=128, output_dim=3, embedding_matrix=embedding_matrix)
                
                label_encoder = LabelEncoder()
                y_train_encoded = label_encoder.fit_transform(y_train)
                y_dev_encoded = label_encoder.transform(y_dev)
                
                # Convert data to PyTorch tensors
                X_train_tensor = torch.tensor(X_train, dtype = torch.long)
                X_dev_tensor = torch.tensor(X_dev, dtype = torch.long)
                y_train_tensor = torch.tensor(y_train_encoded, dtype = torch.long)
                y_dev_tensor = torch.tensor(y_dev_encoded, dtype = torch.long)
                
                train_data = TensorDataset(X_train_tensor, y_train_tensor)
                train_loader = DataLoader(train_data, batch_size=32, shuffle=True)

                criterion = nn.CrossEntropyLoss()
                optimizer = torch.optim.Adam(lstm_model.parameters(), lr=0.001)
                
                num_epochs = 10
                lstm_model.train()

                for epoch in range(num_epochs):
                    for inputs, labels in train_loader:
                        optimizer.zero_grad()
                        output = lstm_model(inputs)
                        loss = criterion(output, labels)
                        loss.backward()
                        optimizer.step()
                    print(f'Epoch {epoch+1}, Loss: {loss.item()}')
                
                lstm_model.eval()

                with torch.no_grad():
                    predictions_dev_lstm = lstm_model(X_dev_tensor)
                    _, predicted_labels = torch.max(predictions_dev_lstm, 1)
                print(classification_report(y_dev_tensor.numpy(), predicted_labels.numpy(), target_names=label_encoder.classes_))
                
                X_test1_tokens = X_test1.apply(lambda x: word_tokenize(x.lower()))

                X_test1_seq = [[word_index[token] for token in tokens if token in word_index] for tokens in X_test1_tokens]
                X_test1_pad = pading_sequences(X_test1_seq, maxlen=max_length)

                y_test1_encoded = label_encoder.transform(y_test1)

                X_test1_tensor = torch.tensor(X_test1_pad, dtype=torch.long)
                y_test1_tensor = torch.tensor(y_test1_encoded, dtype=torch.long)

                lstm_model.eval()

                with torch.no_grad():
                    predictions_test1 = lstm_model(X_test1_tensor)
                    _, predicted_labels_test1 = torch.max(predictions_test1, 1)
                predicted_labels_test1 = [label_encoder.inverse_transform([idx.item()])[0] for idx in predicted_labels_test1]
                predictions_test1_lstm = {str(tweet_id): pred for tweet_id, pred in zip(test1_tweet_ids, predicted_labels_test1)}
                evaluate(predictions_test1_lstm, test1_path, 'LSTM')
                
                X_test2_tokens = X_test2.apply(lambda x: word_tokenize(x.lower()))

                X_test2_seq = [[word_index[token] for token in tokens if token in word_index] for tokens in X_test2_tokens]
                X_test2_pad = pading_sequences(X_test2_seq, maxlen=max_length)

                y_test2_encoded = label_encoder.transform(y_test2)

                X_test2_tensor = torch.tensor(X_test2_pad, dtype=torch.long)
                y_test2_tensor = torch.tensor(y_test2_encoded, dtype=torch.long)

                lstm_model.eval()

                with torch.no_grad():
                    predictions_test2 = lstm_model(X_test2_tensor)
                    _, predicted_labels_test2 = torch.max(predictions_test2, 1)
                predicted_labels_test2 = [label_encoder.inverse_transform([idx.item()])[0] for idx in predicted_labels_test2]
                predictions_test2_lstm = {str(tweet_id): pred for tweet_id, pred in zip(test2_tweet_ids, predicted_labels_test2)}
                evaluate(predictions_test2_lstm, test2_path, 'LSTM')
                
                X_test3_tokens = X_test3.apply(lambda x: word_tokenize(x.lower()))

                X_test3_seq = [[word_index[token] for token in tokens if token in word_index] for tokens in X_test3_tokens]
                X_test3_pad = pading_sequences(X_test3_seq, maxlen=max_length)

                y_test3_encoded = label_encoder.transform(y_test3)

                X_test3_tensor = torch.tensor(X_test3_pad, dtype=torch.long)
                y_test3_tensor = torch.tensor(y_test3_encoded, dtype=torch.long)

                lstm_model.eval()

                with torch.no_grad():
                    predictions_test3 = lstm_model(X_test3_tensor)
                    _, predicted_labels_test3 = torch.max(predictions_test3, 1)
                predicted_labels_test3 = [label_encoder.inverse_transform([idx.item()])[0] for idx in predicted_labels_test3]
                predictions_test3_lstm = {str(tweet_id): pred for tweet_id, pred in zip(test3_tweet_ids, predicted_labels_test3)}
                evaluate(predictions_test3_lstm, test3_path, 'LSTM')


        # Predition performance of the classifiers
        #for testset in testsets:
            #id_preds = {}
            # write the prediction and evaluation code here

            #testset_name = testset
            #testset_path = join('semeval-tweets', testset_name)
            #evaluate(id_preds, testset_path, features + '-' + classifier)

Training maxent
Training maxent


C:\Users\VatsaL\anaconda3\Lib\site-packages\sklearn\linear_model\_logistic.py:458: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


              precision    recall  f1-score   support

    negative       0.68      0.49      0.57       378
     neutral       0.66      0.70      0.68       919
    positive       0.66      0.71      0.68       703

    accuracy                           0.66      2000
   macro avg       0.67      0.63      0.64      2000
weighted avg       0.66      0.66      0.66      2000

C:\Users\VatsaL\NLP Practical\semeval-tweets\twitter-dev-data.txt (MAXENT): 0.584
C:\Users\VatsaL\NLP Practical\semeval-tweets\twitter-test1.txt (MAXENT): 0.186
C:\Users\VatsaL\NLP Practical\semeval-tweets\twitter-test2.txt (MAXENT): 0.271
C:\Users\VatsaL\NLP Practical\semeval-tweets\twitter-test3.txt (MAXENT): 0.235
Training naive_bayes
Training naive_bayes
              precision    recall  f1-score   support

    negative       0.67      0.22      0.34       378
     neutral       0.58      0.75      0.65       919
    positive       0.63      0.61      0.62       703

    accuracy                           0